## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar10_vgg16_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## VGG16 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarVgg16Config(cifar = 10)
model = model_gen.GetCifarVgg16()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 1.727299, Test Acc: 46.96 %
2 epoch end, loss: 1.158102, Test Acc: 60.40 %
3 epoch end, loss: 0.919403, Test Acc: 66.91 %
4 epoch end, loss: 0.772754, Test Acc: 70.70 %
5 epoch end, loss: 0.677590, Test Acc: 76.58 %
6 epoch end, loss: 0.607441, Test Acc: 78.29 %
7 epoch end, loss: 0.553135, Test Acc: 78.86 %
8 epoch end, loss: 0.512411, Test Acc: 77.95 %
9 epoch end, loss: 0.474939, Test Acc: 81.29 %
10 epoch end, loss: 0.446933, Test Acc: 81.73 %
11 epoch end, loss: 0.419743, Test Acc: 83.83 %
12 epoch end, loss: 0.398136, Test Acc: 81.53 %
13 epoch end, loss: 0.382936, Test Acc: 83.49 %
14 epoch end, loss: 0.359570, Test Acc: 83.56 %
15 epoch end, loss: 0.348823, Test Acc: 85.44 %
16 epoch end, loss: 0.331388, Test Acc: 84.60 %
17 epoch end, loss: 0.317979, Test Acc: 83.72 %
18 epoch end, loss: 0.313046, Test Acc: 84.49 %
19 epoch end, loss: 0.297948, Test Acc: 84.70 %
20 epoch end, loss: 0.287019, Test Acc: 85.91 %
21 epoch end, loss: 0.276721, Test Acc: 83.77 %
2

171 epoch end, loss: 0.001108, Test Acc: 92.98 %
172 epoch end, loss: 0.001270, Test Acc: 92.99 %
173 epoch end, loss: 0.001071, Test Acc: 92.89 %
174 epoch end, loss: 0.001092, Test Acc: 92.92 %
175 epoch end, loss: 0.001341, Test Acc: 92.96 %
176 epoch end, loss: 0.001018, Test Acc: 92.98 %
177 epoch end, loss: 0.001061, Test Acc: 92.98 %
178 epoch end, loss: 0.001158, Test Acc: 93.09 %
179 epoch end, loss: 0.000934, Test Acc: 92.97 %
180 epoch end, loss: 0.001052, Test Acc: 93.01 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 93.01 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)